In [ ]:
import pandas as pd
file_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev.parquet"
df = pd.read_parquet(file_path)
df['agent_name'] = 'mem_agent'
df.to_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev_agent_loop.parquet")

In [1]:
import json
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/longmemeval_s_r1_log_no_longcontext_recall20_eval_res.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/longmemeval_s/Qwen3-8B-5k-1k-infty_eval_res.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/processing_log_qwen3-8b_1758104883.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/processing_log_qwen3-8b_faster_1758253073.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/longmemeval/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/longmemeval_oracle/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/ReMe/cookbook/simple_demo/longmemeval/evaluated_unknown.jsonl"
data = [json.loads(line) for line in open(datapath)]

original_datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/data/longmemeval_s.json"
original_data = json.load(open(original_datapath))
for d in data:
    for od in original_data:
        question_id = d.get("question_id", d.get("qid"))
        if "longmemeval_" in question_id:
            question_id = question_id[len("longmemeval_"):-2]
        if od["question_id"] == question_id:
            d["question_type"] = od["question_type"]
            break

# datapath = "longmemeval_s_r1_log_no_longcontext_recall20.json"
# data = json.load(open(datapath))
print("数据条数：" + str(len([x for x in data if x.get('qid') is not None])))
print("正确率：" + str(len([x for x in data if x["metric"].get('llm_score')]) / len([x for x in data if x["metric"].get('llm_score') is not None])))

for tp in ["knowledge-update", "single-session-user", "temporal-reasoning", "single-session-preference", "single-session-assistant", "multi-session"]:
    if len([x for x in data if x.get('question_type')==tp]) == 0:
        print(tp, 0)
    else:
        print(tp, len([x for x in data if x["metric"].get('llm_score') and x.get('question_type')==tp]) / len([x for x in data if x.get('question_type')==tp]))
        print(f"{len([x for x in data if x["metric"].get('llm_score') and x.get('question_type')==tp])} / {len([x for x in data if x.get('question_type')==tp])}")

数据条数：500
正确率：0.382
knowledge-update 0.4358974358974359
34 / 78
single-session-user 0.4142857142857143
29 / 70
temporal-reasoning 0.46616541353383456
62 / 133
single-session-preference 0.6
18 / 30
single-session-assistant 0.17857142857142858
10 / 56
multi-session 0.2857142857142857
38 / 133


In [4]:
import json

datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/locomo/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/locomo_results_qwen3-8b_faster_1759028934.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/locomo_results_1759036272.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/locomo_hippo_qwen_recall42.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/locomo_hippo_qwen_recall20.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/ReMe/cookbook/simple_demo/locomo/evaluated_unknown.jsonl"
data = [json.loads(line) for line in open(datapath)]
# data = sum([d["queries_solutions"] for d in data], start=[])

original_datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/raw/locomo10.json"
original_data = json.load(open(original_datapath))

for d in data:
    for od in original_data:
        if od['sample_id'] in d['qid']:
            q_index = int(d['qid'].split('_')[2])
            d['question_type'] = od['qa'][q_index]['category']

for d in data:
    # d['question_type'] = d['category']
    d["gpt4o_score"] =  d["metric"].get('llm_score')

print("数据条数：" + str(len(data)))
print("正确率：" + str(len([x for x in data if x["gpt4o_score"]]) / len([x for x in data if x["gpt4o_score"] is not None])))

question_type_map = {
    1: "multi hop",
    2: "temporal",
    3: "open domain",
    4: "single hop",
    5: "adversarial"
}


for tp in range(1,6):
    if len([x for x in data if x.get('question_type')==tp]) == 0:
        print(question_type_map[tp], 0)
    else:
        print(question_type_map[tp], len([x for x in data if x["gpt4o_score"] and x.get('question_type')==tp]) / len([x for x in data if x.get('question_type')==tp]))
        print(f"{len([x for x in data if x["gpt4o_score"] and x.get('question_type')==tp])} / {len([x for x in data if x.get('question_type')==tp])}")

数据条数：1986
正确率：0.28751258811681774
multi hop 0.09574468085106383
27 / 282
temporal 0.07476635514018691
24 / 321
open domain 0.21875
21 / 96
single hop 0.06539833531510107
55 / 841
adversarial 0.9955156950672646
444 / 446


In [ ]:
import json
import tqdm
from openai import OpenAI, RateLimitError
import uuid
import backoff
import asyncio
from tqdm.asyncio import tqdm as atqdm

from config import MODEL_NAME, API_CONFIG_LOCAL
from openai import OpenAI

async def gpt4o_evaluation(question, correct_answer, model_response, question_type):
    """
    Use GPT-4o to evaluate the model response
    Based on reprocess_res.py
    """

    API_CONFIG_LOCAL["base_url"] = "http://172.24.139.15:8000/v1"
    openai_client = OpenAI(**API_CONFIG_LOCAL)

    @backoff.on_exception(
        backoff.expo,
        RateLimitError,
        max_tries=16,
        max_time=300,
        jitter=backoff.full_jitter
    )
    async def llm_response(messages):
        # 使用 asyncio.to_thread 将同步调用转为异步
        res = await asyncio.to_thread(
            openai_client.chat.completions.create,
            model=MODEL_NAME,
            messages=messages,
            temperature=0,
            max_tokens=2048,
        )
        response_message = res.choices[0].message.content
        if "</think>" in response_message:
            response_message = response_message.split("</think>")[-1].strip()
        # print(response_message)
        return response_message
    
    def get_anscheck_prompt(task, question, answer, response):
        LLM_TEMPLATE = "I will give you a question, a correct answer, and a response from a model. Please answer yes if the response contains the correct answer. Otherwise, answer no. If the response is equivalent to the correct answer or contains all the intermediate steps to get the correct answer, you should also answer yes. If the response only contains a subset of the information required by the answer, answer no.\n\nQuestion: {}\n\nCorrect Answer: {}\n\nModel Response: {}\n\nIs the model response correct? Answer yes or no only."
        prompt = LLM_TEMPLATE.format(question, answer, response)
        return prompt
    
    async def anscheck(task, question, answer, response):
        prompt = get_anscheck_prompt(task, question, answer, response)
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
        res = await llm_response(messages)
        return 'yes' in res.lower() and not 'no' in res.lower()
    
    return await anscheck(question_type, question, correct_answer, model_response)

async def process_item(d, output_file, semaphore, file_lock):
    async with semaphore:
        score = await gpt4o_evaluation(d["input"], d["answer"], d["response"], d["question_type"])
        d["gpt4o_score"] = score
        
        # 使用文件锁确保并发写入安全
        async with file_lock:
            with open(output_file, "a") as f:
                f.write(json.dumps(d) + '\n')
        
        return d

async def main():
    data_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty.jsonl"
    data = [json.loads(line) for line in open(data_path)]
    
    output_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty_.jsonl"
    
    # 创建信号量控制并发数
    semaphore = asyncio.Semaphore(20)  # 可调整并发数
    
    # 创建文件锁，确保并发写入安全
    file_lock = asyncio.Lock()
    
    # 创建所有任务
    tasks = [process_item(d, output_file, semaphore, file_lock) for d in data]
    
    # 使用异步进度条执行
    results = await atqdm.gather(*tasks, desc="Processing")

if __name__ == "__main__":
    # asyncio.run(main())
    await main()

In [2]:
from config import MODEL_NAME, API_CONFIG_LOCAL, API_CONFIG
from openai import OpenAI
API_CONFIG_LOCAL["base_url"] = "http://172.24.139.15:8000/v1"
openai_client = OpenAI(**API_CONFIG_LOCAL)
openai_client = OpenAI(**API_CONFIG)
messages = [
    {
        "role": "user",
        "content": "hello"
    }
]
res = openai_client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0,
    max_tokens=2048,
    logprobs=1
)
print(res)

ChatCompletion(id=None, choices=None, created=None, model=None, object=None, service_tier=None, system_fingerprint=None, usage=None, code=100049, msg='接口验证失败,请检查Token是否正确', success=False, data={})


In [ ]:
[x.token for x in res.choices[0].logprobs.content]
res.choices[0].message.content

['<think>',
 '\n',
 'Okay',
 ',',
 ' the',
 ' user',
 ' said',
 ' "',
 'hello',
 '".',
 ' I',
 ' need',
 ' to',
 ' respond',
 ' appropriately',
 '.',
 ' Since',
 ' it',
 "'s",
 ' a',
 ' greeting',
 ',',
 ' I',
 ' should',
 ' acknowledge',
 ' it',
 ' and',
 ' offer',
 ' assistance',
 '.',
 ' Let',
 ' me',
 ' make',
 ' sure',
 ' the',
 ' response',
 ' is',
 ' friendly',
 ' and',
 ' open',
 '-ended',
 '.',
 ' Maybe',
 ' ask',
 ' how',
 ' I',
 ' can',
 ' help',
 ' them',
 ' today',
 '.',
 ' Keep',
 ' it',
 ' simple',
 ' and',
 ' welcoming',
 '.\n',
 '</think>',
 '\n\n',
 'Hello',
 '!',
 ' How',
 ' can',
 ' I',
 ' assist',
 ' you',
 ' today',
 '?',
 ' �',
 '�',
 '<|im_end|>']

In [1]:
import json
def load_context() -> str:
    """Load contextual information fed into ``ToolMemoryAgentLoop``.

    Projects can override this helper to supply domain-specific context. By default it
    returns the value from ``TOOL_MEM_AGENT_TEST_CONTEXT`` (or an empty string).
    """

    # return os.environ.get("TOOL_MEM_AGENT_TEST_CONTEXT", "I like to eat bananas.")
    raw = json.load(open("/mnt/pfs-guan-ssai/nlu/zhangkehao/zep/benchmarks/longmemeval/data/longmemeval_s.json"))
    item0 = raw[0]
    context = "Below is a conversation between User and Assistant.\n\n"
    for idx, session in enumerate(item0["haystack_sessions"]):
        session_date = item0["haystack_dates"][idx]
        session_conv = ''
        for turn in session:
            role = "User" if turn['role'] == 'user' else "Assistant"
            turn_text = f'{role} said, "{turn["content"]}"'
            turn_text += '\n'
            session_conv += turn_text
        if not session_conv:
            session_conv = "NO CONVERSATION"
        query_conv = f'DATE: {session_date}\nCONVERSATION:\n{session_conv}\n\n'
        context += query_conv
    return context

In [ ]:
# reformat
import json
input_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/base_result.jsonl"
output_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/base_result_reformat.jsonl"
original_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/zep/benchmarks/longmemeval/data/longmemeval_s.json"
data = [json.loads(line) for line in open(input_file)]
original_data = json.load(open(original_file))
with open(output_file, "w") as f:
    for d in data:
        original_d = original_data[d['idx']]
        response = d["response"]
        if "\\boxed{" in response and "}" in response.split("\\boxed{")[-1]:
            response = response.split("\\boxed{")[-1]
            response = response[:response.rindex("}")]
        new_d = {
            "qid": "longmemeval_" + original_d["question_id"] + "_0",
            "query": original_d["question"],
            "expected_answer": original_d["answer"],
            "response": response
        }
        f.write(json.dumps(new_d) + '\n')

In [30]:
import pandas as pd
import numpy as np
memalpha_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl.parquet'
locomo_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/locomo_train_verl.parquet'
hpq_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_train_mem_agent_loop_chunk2000.parquet'
for input_file, name in zip([memalpha_file, locomo_file, hpq_file], ['memalpha', 'locomo', 'hotpotqa']):
    print(name)
    df = pd.read_parquet(input_file)
    print(np.mean([len([len(x) for x in y['tools_kwargs']['memory_bm25_retrieve']['create_kwargs']['chunks']]) for y in df['extra_info']]), 'chunks') # 平均chunks数量
    print(np.mean([len(x) for y in df['extra_info'] for x in y['tools_kwargs']['memory_bm25_retrieve']['create_kwargs']['chunks']]), 'chars\n') # 平均每个chunk字符数量

memalpha
9.89596167008898 chunks
4046.8266011896526 chars

locomo
19.0 chunks
3942.8947368421054 chars

hotpotqa
61.06103515625 chunks
1701.0528355977065 chars



In [2]:
import json
import numpy as np
hotpotqa_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_hotpotqa_200.json'
locomo_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_locomo.json'
longmemeval_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_longmemeval.json'
msc_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_msc_batch5.json'
for input_file, name in zip([hotpotqa_file, locomo_file, longmemeval_file, msc_file], ['hotpotqa', 'locomo', 'longmemeval', 'msc']):
    print(name)
    data = json.load(open(input_file))
    print(np.mean([len(item['chunks']) for item in data]), 'chunks') # 平均chunks数量
    print(np.mean([len(chunk) for item in data for chunk in item['chunks']]), 'chars\n') # 平均每个chunk字符数量

hotpotqa
200.0 chunks
570.0202734375 chars

locomo
27.2 chunks
3398.1507352941176 chars

longmemeval
47.764 chunks
10521.834394104346 chars

msc
25.0 chunks
1563.9144 chars



In [80]:
def calculate_value(row):
    row['extra_info']['num_chunks'] = row['extra_info'].pop('num_docs')
    row['extra_info']['question'] = [row['extra_info']['question']]
    return row
df = df.apply(calculate_value, axis=1)
df.to_parquet(output_file)

In [5]:
df['agent_name'] = 'tool_mem_agent'
def get_chunks(context_text, chunk_size):
    separator = "\n"
    small_chunks = context_text.split(separator)
    chunks = []
    to_append = ""
    for chunk in small_chunks:
        if len(to_append) + len(chunk) + len(separator) <= chunk_size:
            if to_append:
                to_append += separator + chunk
            else:
                to_append = chunk
        else:
            if to_append:
                chunks.append(to_append)
            to_append = chunk
    if to_append:
        chunks.append(to_append)
    return chunks

def calculate_value(row):
    idx = row['extra_info']['index']
    filename = f"/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/memagent/store/memory_store_val{idx}.jsonl"
    chunks = get_chunks(row['context'], 2000)
    row['extra_info']["tools_kwargs"] = {
        "memory_add": {
            "create_kwargs": {"filename": filename}
        },
        "memory_update": {
            "create_kwargs": {"filename": filename}
        },
        "memory_delete": {
            "create_kwargs": {"filename": filename}
        },
        "memory_key_retrieve": {
            "create_kwargs": {"filename": filename}
        },
        "memory_list": {
            "create_kwargs": {"filename": filename}
        },
        "memory_bm25_retrieve": {
            "create_kwargs": {"chunks": chunks}
        },
        "memory_embedding_retrieve": {
            "create_kwargs": {"chunks": chunks}
        },
    }
    return row['extra_info']

df['extra_info'] = df.apply(calculate_value, axis=1)
df.to_parquet(output_file)